In [20]:
import pandas as pd
import numpy as np

In [14]:
df_base = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/communes_AEP_2023.csv", encoding="Windows-1252", sep=";", low_memory=False)

In [15]:
df_base

,DPT de la commune,Nom de la commune adhérente,Identifiant SISPEA de la commune adhérente,Code INSEE de la commune adhérente,Population de la commune,Type d'adhésion,Nom de la collectivité via laquelle la commune adhère,N° SIREN de la collectivité via laquelle la commune adhère,Identifiant SISPEA de la collectivité via laquelle la commune adhère,Nom de la collectivité de l'entité de gestion à laquelle la commune adhère,...,Zone tarifaire 1,Tarif zone tarifaire 1,Zone tarifaire 2,Tarif zone tarifaire 2,Zone tarifaire 3,Tarif zone tarifaire 3,Zone tarifaire 4,Tarif zone tarifaire 4,Zone tarifaire 5,Tarif zone tarifaire 5
0,01,Cerdon,348,01068,788.0,Adhérent direct,NaN,NaN,NaN,Cerdon,...,Indicateur tarif,"3,64",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,Tossiat,672,01422,1390.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,Dompierre-sur-Veyle,418,01145,1214.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,La Tranclière,675,01425,291.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,Montagnat,515,01254,2193.0,Adhérent direct,NaN,NaN,NaN,SIE Ain Veyle Revermont,...,Indicateur tarif,"2,72",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33901,93,Bondy,36616,93010,53509.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33902,93,Noisy-le-Sec,36634,93053,45156.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33903,93,Pantin,36635,93055,60597.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33904,93,Le Pré-Saint-Gervais,36638,93061,17342.0,Adhérent direct,NaN,NaN,NaN,Est Ensemble (T8),...,Indicateur tarif,"1,38",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Filtrage du df pour récupérer les communes que l'on veut
df_base = df_base[df_base['DPT de la commune'].isin(['75', '77', '78', '91', '92', '93', '94', '95'])][["DPT de la commune", "Nom de la commune adhérente", "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère"]]

In [18]:
df_base

,DPT de la commune,Nom de la commune adhérente,Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère
9415,78,Dannemarie,174831
9435,78,Mondreville,175062
15894,77,Bransles,67568
21710,95,Ronquerolles,167404
29146,77,Hautefeuille,233974
...,...,...,...
33901,93,Bondy,198916
33902,93,Noisy-le-Sec,198916
33903,93,Pantin,198916
33904,93,Le Pré-Saint-Gervais,198916


In [41]:
dico = {"annee" : [], "lignes" : [], "NaN" : []}


for annee in range(2012, 2023):
    dico["annee"].append(annee)
    df = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/tarifs_AEP_" + str(annee) + ".csv", encoding="Windows-1252", sep=";", low_memory=False)
    
    # Modification des valeurs de la colonn id SISPEA pour futur merge
    df['Id SISPEA de la collectivité'] = df['Id SISPEA de la collectivité'].str.replace('CollectivityId=', '').astype(int)

    # On ne garde donc que cette colonne et la D102.0:
    df_clean = df[['Id SISPEA de la collectivité', 'D102.0']]

    df_test = pd.merge(df_base, df_clean, how = 'left', left_on = "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", right_on = "Id SISPEA de la collectivité").drop(["Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", "Id SISPEA de la collectivité"], axis = 1)

    dico["NaN"].append(len(df_test[df_test['D102.0'].isna() == True]))
    dico["lignes"].append(len(df_test))

df_final = pd.DataFrame(dico)
df_final["%NaN"] = round(df_final['NaN']/df_final["lignes"] *100 , 2)

df_final

,annee,lignes,NaN,%NaN
0,2012,1411,972,68.89
1,2013,1523,771,50.62
2,2014,1537,749,48.73
3,2015,1481,881,59.49
4,2016,3924,777,19.80
5,2017,5437,1698,31.23
6,2018,5105,1168,22.88
7,2019,8196,2319,28.29
8,2020,10464,865,8.27
9,2021,10707,853,7.97


In [22]:

df_list = []

for annee in range(2012, 2024):
    df = pd.read_csv("C:/Users/Antoine/Documents/AquaVision/donnees/brutes/eau_potable/tarifs_AEP_" + str(annee) + ".csv", encoding="Windows-1252", sep=";", low_memory=False)
    
    df['Id SISPEA de la collectivité'] = df['Id SISPEA de la collectivité'].str.replace('CollectivityId=', '').astype(int)

    df = df[['Id SISPEA de la collectivité', 'D102.0']]    

    df_temp = pd.merge(df_base, df, how = 'left', left_on = "Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", right_on = "Id SISPEA de la collectivité").drop(["Identifiant SISPEA de la collectivité de l'entité de gestion à laquelle la commune adhère", "Id SISPEA de la collectivité"], axis = 1)

    
    df_temp['D102.0'] = df_temp['D102.0'].str.replace(",", ".").astype(float)

    df_temp = df_temp.fillna(0)

    df_temp = df_temp.groupby('Nom de la commune adhérente').max()

    df_temp['Année'] = annee

    df_list.append(df_temp)

# Concatenation de tous les DataFrames accumulés dans la liste
df_prix = pd.concat(df_list)

df_prix['D102.0'] = df_prix['D102.0'].apply(lambda x : np.nan if x ==0 else round(x , 2))

df_prix





,DPT de la commune,D102.0,Année
Nom de la commune adhérente,,,
Abbéville-la-Rivière,91,NaN,2012
Ableiges,95,NaN,2012
Ablis,78,2.37,2012
Ablon-sur-Seine,94,2.05,2012
Achères,78,NaN,2012
...,...,...,...
Étréchy,91,5.16,2023
Étrépilly,77,2.60,2023
Évecquemont,78,3.64,2023


In [27]:
df_prix.to_csv("C:/Users/Antoine/Documents/AquaVision/donnees/nettoyees/tarifs_eau_potable.csv", index = False)